# Comparing MASH analysis with simple multivariate analysis

Previously we showed that even though univariate analysis with degenerated MASH model gives identical results to SuSiE as expected (with non-decreasing ELBO), in multivariate calculations the ELBO does not always non-decreasing.

To investigate the issue we will 1) further simplify the problem and 2) isolate the problem to posterior calculation versus ELBO calculations and check which part is problematic. The best way to achieve both is to implement a simple Bayesian multivariate regression model with prior $b \sim MVN(0, U)$ where $U$ is known, instead of using MASH prior for $b$.

This feature is now implemented in the [`BayesianMultivariateRegression` class](https://github.com/gaow/mmbr/blob/master/R/bayesian_mulvariate_regression.R) with an interface added to the main function such that the code will be triggered [when prior variance is a matrix](https://github.com/gaow/mmbr/blob/dccc26cfbba0cfd3e6f20b37e06d4db6305b834f/R/main.R#L110).

## Test the calculation agree with univariate code

In [1]:
library(mmbr)
set.seed(2)
L = 5

Loading required package: mashr
Loading required package: ashr


In [2]:
dat = mmbr_sim1(r=1)

Run the simulated univariate data with SuSiE,

In [3]:
res = susieR::susie(dat$X,dat$y,L=L,scaled_prior_variance=0.2,estimate_residual_variance=F,estimate_prior_variance=F)
res$elbo

[1] -429.4050 -409.5493 -407.7077 -407.7058 -407.7058

Now run it with multivariate simple prior implementation in `mmbr`,

In [4]:
res = mmbr::susie(dat$X,dat$y,L=L,V=0.2*var(dat$y),compute_objective=T,estimate_residual_variance=F,estimate_prior_variance=F)
res$elbo

[1] -429.4050 -409.5493 -407.7077 -407.7058 -407.7058

So it is confirmed that this implementation produces identical results as SuSiE runs.

# Test multivariate analysis

In [5]:
set.seed(2)
dat = mmbr_sim1(r=5)

In [7]:
dat$V

1.14068826,-0.02561830,-0.19275481,-0.040421229,-0.029376381
-0.02561830,1.01088663,0.03491952,0.018064842,0.016833489
-0.19275481,0.03491952,1.15215509,-0.112673234,-0.063194855
-0.04042123,0.01806484,-0.11267323,0.836044978,0.008248383
-0.02937638,0.01683349,-0.06319485,0.008248383,0.865937437


In [6]:
res = mmbr::susie(dat$X,dat$y,L=L,V=dat$V,compute_objective=T,estimate_residual_variance=F,estimate_prior_variance=F)
res$elbo

[1] -2184.076 -2173.199 -2169.551 -2167.744 -2167.267 -2167.087 -2166.514
 [8] -2166.013 -2164.894 -2163.635 -2163.574 -2163.573 -2163.573

Here the ELBO is non-decreasing.